In [1]:
from __future__ import print_function

import sys
import os
import time

import numpy as np
import theano
import theano.tensor as T

import lasagne

from kfkd import load2d, makeSubmit
from utils import iterate_minibatches

# Load the dataset
print("Loading data...")

#Attention! Not all data is being loaded, only the rows without N/A vals.
X, y = load2d()
X_train = X[:1900]
y_train = y[:1900]
X_test = X[1900:]
y_test = y[1900:]
print('Done')

Using gpu device 0: GeForce GTX 970 (CNMeM is disabled, cuDNN 5005)


Loading data...
Done


In [2]:
def build_cnn(input_var=None):

    # Input layer:
    network = lasagne.layers.InputLayer(shape=(None, 1, 96, 96),
                                        input_var=input_var)
    
    # Convolutional layer with 32 kernels of size 5x5. Strided and padded
    # convolutions are supported as well; see the docstring.
    network = lasagne.layers.Conv2DLayer(
            network, num_filters=16, filter_size=(5, 5),
            nonlinearity=lasagne.nonlinearities.rectify,
            W=lasagne.init.GlorotUniform())

    # Max-pooling layer of factor 2 in both dimensions:
    network = lasagne.layers.MaxPool2DLayer(network, pool_size=(2, 2))

    # Another convolution with 32 5x5 kernels, and another 2x2 pooling:
    network = lasagne.layers.Conv2DLayer(
            network, num_filters=16, filter_size=(5, 5),
            nonlinearity=lasagne.nonlinearities.rectify)
    network = lasagne.layers.MaxPool2DLayer(network, pool_size=(2, 2))

    # A fully-connected layer of 256 units with 50% dropout on its inputs:
    network = lasagne.layers.DenseLayer(
            lasagne.layers.dropout(network, p=0.5),
            num_units=256,
            nonlinearity=lasagne.nonlinearities.rectify)

    # And, finally, the 10-unit output layer with 50% dropout on its inputs:
    network = lasagne.layers.DenseLayer(
            lasagne.layers.dropout(network, p=0.5),
            num_units=30,
            nonlinearity=None)

    return network

num_epochs=100

# Prepare Theano variables for inputs and targets
input_var = T.tensor4('inputs')
target_var = T.dmatrix('targets')

# Create neural network model (depending on first command line parameter)
print("Building model and compiling functions...")
network = build_cnn(input_var)

# Create a loss expression for training, i.e., a scalar objective we want
# to minimize (for our multi-class problem, it is the cross-entropy loss):
prediction = lasagne.layers.get_output(network)
loss = lasagne.objectives.squared_error(prediction, target_var)
loss = loss.mean()
# We could add some weight decay as well here, see lasagne.regularization.

# Create update expressions for training, i.e., how to modify the
# parameters at each training step. Here, we'll use Stochastic Gradient
# Descent (SGD) with Nesterov momentum, but Lasagne offers plenty more.
params = lasagne.layers.get_all_params(network, trainable=True)
updates = lasagne.updates.nesterov_momentum(
        loss, params, learning_rate=0.1, momentum=0.9)

# Create a loss expression for validation/testing. The crucial difference
# here is that we do a deterministic forward pass through the network,
# disabling dropout layers.
test_prediction = lasagne.layers.get_output(network, deterministic=True)
test_loss = lasagne.objectives.squared_error(test_prediction, target_var)
test_loss = test_loss.mean()

# Compile a function performing a training step on a mini-batch (by giving
# the updates dictionary) and returning the corresponding training loss:
train_fn = theano.function([input_var, target_var], loss, updates=updates)

# Compile a second function computing the validation loss and accuracy:
val_fn = theano.function([input_var, target_var], [test_loss])

# Compile function for prediction for test dataset
test_fn = theano.function([input_var], [test_prediction])

# Finally, launch the training loop.
print("Starting training...")
# We iterate over epochs:
for epoch in range(num_epochs):
    # In each epoch, we do a full pass over the training data:
    train_err = 0
    train_batches = 0
    start_time = time.time()
    for batch in iterate_minibatches(X_train, y_train, 128, shuffle=True):
        inputs, targets = batch
        train_err += train_fn(inputs, targets)
        train_batches += 1

    # And a full pass over the validation data:
    val_err = val_fn(X_test, y_test)[0]

    # Then we print the results for this epoch:
    print("Epoch {} of {} took {:.3f}s".format(epoch + 1, num_epochs, time.time() - start_time))
    print("  training loss:\t\t{:.6f}".format(train_err / train_batches))
    print("  validation loss:\t\t{:.6f}".format(val_err / 1.0))

Building model and compiling functions...
Starting training...
Epoch 1 of 100 took 0.603s
  training loss:		0.058388
  validation loss:		0.007349
Epoch 2 of 100 took 0.571s
  training loss:		0.025784
  validation loss:		0.006679
Epoch 3 of 100 took 0.574s
  training loss:		0.009203
  validation loss:		0.005314
Epoch 4 of 100 took 0.578s
  training loss:		0.006993
  validation loss:		0.004865
Epoch 5 of 100 took 0.569s
  training loss:		0.006460
  validation loss:		0.004730
Epoch 6 of 100 took 0.564s
  training loss:		0.006242
  validation loss:		0.004900
Epoch 7 of 100 took 0.562s
  training loss:		0.005932
  validation loss:		0.004779
Epoch 8 of 100 took 0.562s
  training loss:		0.005769
  validation loss:		0.004778
Epoch 9 of 100 took 0.569s
  training loss:		0.005603
  validation loss:		0.004750
Epoch 10 of 100 took 0.566s
  training loss:		0.005521
  validation loss:		0.004777
Epoch 11 of 100 took 0.563s
  training loss:		0.005382
  validation loss:		0.004763
Epoch 12 of 100 took 0

In [4]:
#Load data for for submission
X_submit, _ = load2d(test=True)

#Make predictions with trained network
preds = test_fn(X_submit)[0]

#Create ./submission.csv that you should upload to Kaggle.com to get leaderboard results
makeSubmit(preds)